In [2]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
from utils.point_cloud_util import load_labels, colorize_point_cloud
import multiprocessing

In [3]:
sparse_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/sparse"
dense_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/dense"
gt_dir = "/home/ylao/data/semantic3d"

In [4]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

# Sparse labels
sparse_labels = np.array(load_labels(os.path.join(sparse_dir, file_prefix + ".labels")), dtype=np.int32)

# Sparse points
sparse_pcd = open3d.read_point_cloud(os.path.join(sparse_dir, file_prefix + ".pcd"))
sparse_points = np.asarray(sparse_pcd.points)

# Dense points
dense_pcd = open3d.read_point_cloud(os.path.join(gt_dir, file_prefix + ".pcd"))
dense_points = np.asarray(dense_pcd.points)

# Ground-trugh dense labels
gt_labels = load_labels(os.path.join(gt_dir, file_prefix + ".labels"))

In [5]:
# Build KNN tree
sparse_pcd_tree = open3d.KDTreeFlann(sparse_pcd)

In [8]:
def match_knn_label(dense_index):
    global dense_points
    global sparse_labels
    global sparse_pcd_tree
    
    dense_point = dense_points[dense_index]
    k, sparse_indexes, _ = sparse_pcd_tree.search_hybrid_vector_3d(dense_point, 0.1, 10)
    assert k == len(sparse_indexes)
    if k == 0:
        k, sparse_indexes, _ = sparse_pcd_tree.search_knn_vector_3d(dense_point, 10)
    knn_sparse_labels = sparse_labels[sparse_indexes]
    dense_label = np.bincount(knn_sparse_labels).argmax()
        
    return dense_label

# Assign labels
start = time.time()
dense_indexes = list(range(len(dense_points)))
with multiprocessing.Pool() as pool:
    dense_labels = pool.map(match_knn_label, dense_indexes)
print("knn match time: ", time.time() - start)

# Eval
cm = ConfusionMatrix(9)
cm.increment_from_list(gt_labels, dense_labels)
cm.print_metrics()

knn match time:  46.46926927566528
Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14479      74    1839     995    9113      70   52799     150 
          1       0 13144262    7132     768   33640  464577    6930    1837    3927 
          2       0   21123    3051     790    5037     965    1640     110     240 
          3       0    3017     305  216831   30226   23990     973     893      82 
          4       0    7612   10859  199210  898252   87390  106303     358   16068 
          5       0  413004     907   22801    2726 8270006   22277   12880   11340 
          6       0  131017    1330   15411  154632 2744521  337516  106714    8596 
          7       0   13582       0     370     285     220     241   71716      68 
          8       0   17930       0       0     826   28731    1447   22906  185512 
IoU per class:
[0.9210884548236281,
 0.057039765185365215,
 0.4204864767378948,
 0.5782400683908